# Pandas part 3: Grouping data and merging tables

Notebook developed by Sam Maurer

In [ ]:
import pandas as pd

## 1. Loading the data

In this demo we'll use a set of data about buildings and households in San Francisco. It's in [HDF format](https://en.wikipedia.org/wiki/Hierarchical_Data_Format), which is a way of storing data on disk that similar to a database.

I'm having trouble loading this data directly from a URL, so instead we'll have Python download the file and save it into the same folder as this notebook.

In [ ]:
url = 'https://github.com/urbansim/urbansim_parcels/raw/master/sf_example/data/sanfran_public.h5'

In [ ]:
import requests  #  third-party library for making HTTP requests
r = requests.get(url)  # gets everything (data + metadata) that a web browser would
open('sanfran_public.h5', 'wb').write(r.content)  # saves content to disk

An HDF file can contain more than one table. Let's see what they are:

In [ ]:
hdf = pd.HDFStore('sanfran_public.h5')
print(hdf.keys())
hdf.close()

We can load these tables as DataFrames one at a time.

### New function -- `pd.read_hdf()`, documentation [here](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.read_hdf.html)
- arguments: a filename or URL, and the name of a table inside the file
- returns: a DataFrame

In [ ]:
buildings = pd.read_hdf('sanfran_public.h5', 'buildings')

In [ ]:
buildings.head()

Some things to notice:

1. Unlike when we read a CSV, the DataFrame's index now has a **name**. And its values are **not sequential**. This was set up before the data was saved, and the HDF format preserves it.

  DataFrame indexes are different from normal columns. In some ways you can treat them like regular data, but often not. To remove the index's special properties, you can copy the values into a separate column, or run `.reset_index()` on the DataFrame.
  
  
  
2. Does anything strike you about the data values in the table?

### Review

Try using `.describe()` to get some descriptive statistics about the buildings.

Then use `.plot.hist()` to make histograms of some of the variables.

Remember that you can filter data using `.loc[<expression>]` to make the histogram clearer!

In [ ]:
# Run this cell to get rid of the scientific notation

pd.options.display.float_format = '{:20,.2f}'.format

## 2. Grouping data

Often you'll want to divide your data into categories while analyzing it. You can always do this manually with filter expressions, but Pandas also has tools to do it automatically.

<img src='https://raw.githubusercontent.com/waddell/CP255/master/05-data-manipulation/img/groupby.png' height="800" width="500">

### New function -- `.groupby()`, documentation [here](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.groupby.html)
- call from: a DataFrame
- arguments: name of a column representing groups
- returns: a special "grouped" DataFrame

In [ ]:
buildings.loc[buildings.stories<7].groupby('stories').residential_units.describe()

Let's review that syntax piece by piece:

- `buildings` --> a DataFrame
- `buildings.loc[buildings.stories<7]` --> only rows with <7 stories
- `.groupby('stories')` --> divide the data into groups for further analysis
- `.residential_units` --> focus on a single column
- `.describe()` --> descriptive statistics broken down by group

### Exercise

What years were buildings of different heights constructed? All you need to do is replace one of the column names from the previous line of code.

Now, use this line of code to add a column listing the decade of construction: 

`buildings['decade'] = (buildings.loc[buildings.year_built<2010].year_built//10)*10`

Re-group the data using the new variable. What can you learn about what's been built over time?

## 3. Merging data

Sometimes you'll want to merge information together from more than one DataFrame. This is a common operation when data is divided into multiple, relational tables like in a database.

The best way to start is to think about the hierarchy of the data tables you're working with. For example:

- multiple *households* go into a *building*
- multiple *buildings* go into a *census tract*
- multiple *census tracts* go into a *county*

The most common type of merge looks like this: For each row of data about the **finer-grained thing**, use an ID to look up information about the associated **coarser thing**, and add the information as new columns.

### New function -- `pd.merge()`, documentation [here](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.merge.html)
- arguments: two source DataFrames, ids to use for joining, instructions on how to merge
- returns: a new DataFrame

### Loading more data: households

To try this out, we'll load a second table with data about households in San Francisco. Note that these are **not real households**! The data is what's called a "synthetic population", where individual households are created algorithmically to match the known demographic characteristics of each census tract. In aggregate, the association between people and buildings will be realistic, but the individual data points are fake.

In [ ]:
households = pd.read_hdf('sanfran_public.h5', 'households')

In [ ]:
households.head()

### Merging coarse-to-fine (one-to-many)

Multiple **households** are in each **building**. And we see that each household has a `building_id` associated with it. 

This means that we're able to merge characteristics of the **buildings** directly onto the **households** table.

In [ ]:
extended_households = pd.merge(households, buildings, on='building_id', how='left')

In [ ]:
extended_households.head()

### Merging fine-to-coarse (many-to-one)

What if we want to bring information from the **households** table over to the **buildings** table? This is harder! 

Because there are multiple households in each building, we have to **summarize** the information first: calculate a sum, or a mean, or even just pick the first value.

But **`.groupby()`** can help us with this:

In [ ]:
household_totals = households.groupby('building_id').median()

In [ ]:
extended_buildings = pd.merge(buildings, household_totals, on='building_id', how='left')

In [ ]:
extended_buildings.head()

### Exercise!

What can you learn from the merged data? Here are some things you might look at:

- How do people's incomes vary by housing tenure? ("Tenure" means rent/own in this context.) Can you use this to guess which tenure category is which?
  

- How does the tenure split vary by decade of construction?
  

- Do higher-income households tend to live in larger buildings or in smaller buildings?
  

- What's the total population in single-family homes vs. multi-unit buildings? (You'll need to summarize the households with `.sum()` to count the people.)
  

- How has the square footage of units changed over time?